In [2]:
from problog.program import PrologString
from problog import get_evaluatable
import os# define the arguments for the model:

def get_model_files(date:str, model_name:str):
    model_files = []
    for root, dirs, files in os.walk(os.path.join("src/generated/models", date)):
        for file in files:
            if file.startswith("prolog_model_"+model_name+"_"):
                model_files.append(os.path.join(root, file))

    models_string_dir = {}
    for file in model_files:
        with open(file, 'r') as f:
            models_string_dir[file] = f.read()
    print(f"Found {len(models_string_dir)}")
    return models_string_dir

def get_single_model_file_with_path(model_path:str):

    if os.path.isfile(model_path):
        with open(model_path, 'r') as f:
            model_string = f.read()
    else:
        print(f"this file could not be found")
    return model_string

def prolog_reasoning(model:str):
    result = get_evaluatable().create_from(PrologString(model)).evaluate()
    print("% ProbLog Inference Result：")
    for query_key, probability in result.items():
        print(f"{query_key} = {probability:.4f}")

In [10]:
model = """
zebra(Houses) :-
 houses(Houses),
 
member(house(englishman, _, _, red, _), Houses),
member(house(spaniard, dog, _, _, _), Houses),
member(house(_, _, coffee, green, _), Houses),
member(house(ukrainian, _, tea, _, _), Houses),
right_of(house(_, _, _, green, _), house(_, _, _, ivory, _), Houses),
member(house(_, snails, _, _, winston), Houses),
member(house(_, _, _, yellow, kools), Houses),
Houses = [_, _, house(_, _, milk, _, _), _, _],
Houses = [house(norwegian, _, _, _, _)|_],
next_to(house(_, _, _, _, chesterfields), house(_, fox, _, _, _), Houses),
next_to(house(_, _, _, _, kools), house(_, horse, _, _, _), Houses),
member(house(_, _, orange_juice, _, lucky_strikes), Houses),
member(house(japanese, _, _, _, parliaments), Houses),
next_to(house(norwegian, _, _, _, _), house(_, _, _, blue, _), Houses),
member(house(_, zebra, _, _, _), Houses),
member(house(_, _, water, _, _), Houses),
houses([
 house(_, _, _, _, _),
 house(_, _, _, _, _),
 house(_, _, _, _, _),
 house(_, _, _, _, _),
 house(_, _, _, _, _)
]).
right_of(A, B, [B, A | _]).
right_of(A, B, [_ | Y]) :- right_of(A, B, Y).
next_to(A, B, [A, B | _]).
next_to(A, B, [B, A | _]).
next_to(A, B, [_ | Y]) :- next_to(A, B, Y).
my_member(X, [X|_]).
my_member(X, [_|Y]) :- my_member(X, Y).
query(zebra(Houses)).
"""

prolog_reasoning(model)

UnknownClause: No clauses found for 'houses/1' at 3:2.

/Users/zhenzhili/miniforge3/envs/langda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
import faiss
from typing import List
from config import paths
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import BaseDocumentTransformer, Document

from langchain.document_loaders import PyMuPDFLoader

from langchain_huggingface import HuggingFaceEmbeddings

problog_official_doc_path = paths.get_absproj_path("problog-readthedocs-io-en-latest.pdf")
loader = PyMuPDFLoader(problog_official_doc_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# Initialize Ollama embeddings using DeepSeek-R1
from concurrent.futures import ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
embedding_function = OllamaEmbeddings(model="nomic-embed-text")

# # Parallelize embedding generation
# def generate_embedding(chunk:Document) -> List[float]:
#     return embedding_function.embed_query(chunk)

# with ThreadPoolExecutor() as executor:
#     embeddings = list(executor.map(generate_embedding, chunks))

#  Abstract structure for an index, supports adding vectors and searching them.
# All vectors provided at add or search time are 32-bit float arrays, although the internal representation may vary.
# index = faiss.IndexFlatL2(len(embedding_function.embed_query("hello world"))) # Input: vector dimension

vector_store = FAISS.from_documents(chunks, embedding_function)

vector_store.save_local(paths.get_absproj_path("faiss_langda_kb"))
# vector_store = FAISS(
#     embedding_function=embedding_function,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

# from uuid import uuid4
# from langchain_core.documents import Document

# uuids = [str(uuid4()) for _ in range(len(chunks))]


In [ ]:
from config import paths
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings

problog_official_doc_path = paths.get_absproj_path("problog-readthedocs-io-en-latest.pdf")
loader = PyMuPDFLoader(problog_official_doc_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

embedding_function = OllamaEmbeddings(model="nomic-embed-text")

vector_store = FAISS.from_documents(chunks, embedding_function)

vector_store.save_local(paths.get_absproj_path("faiss_langda_kb"),index_name="problog_readthedocs")


In [14]:
import re
from typing import Literal, List
from config import paths
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings

# ======================================================================== #
#                                Retriever                                 #
# ======================================================================== #
class LangdaVectorStore:
    def __init__(self, 
            langda_db_name:Literal["Prolog_builtins", "Problog_builtins", "Available_libraries"]):
        self.problog_official_doc_path = paths.get_absproj_path("problog-readthedocs-io-en-latest.pdf")
        self.langda_db_dir = paths.get_absproj_path("faiss_langda_kb")
        self.index_name=langda_db_name
        self.index_file = self.langda_db_dir / f"{self.index_name}.faiss"
        self.embedding_function = OllamaEmbeddings(model="nomic-embed-text")
        # Enhanced section metadata with descriptions and keywords Supported_and_unsupported_Prolog_builtins ProbLog_specific_builtins Available_libraries
        self.section_metadata = {
            '4.1': {
                'title': 'Prolog_builtins',
                'description': '''Standard Prolog predicates supported or unsupported by ProbLog (based on Yap Prolog):
            - Control flow (true, fail, conjunction, disjunction, negation, cut, conditional statements, etc.)
            - Term manipulation (var, atom, functor, arg, etc.)
            - Arithmetic operations (+, -, *, /, and other mathematical functions)
            - Term comparison (==, \\==, @<, sort, etc.)
            - other builtins...''',
                'keywords': ['prolog', 'builtin', 'predicate', 'standard', 'yap'],
                'summary': 'This section covers standard Prolog predicates that are supported in ProbLog, including control predicates, term manipulation, arithmetic operations, and more.'
            },
            '4.2': {
                'title': 'Problog_builtins',
                'description': '''Special predicates unique to ProbLog for probabilistic logic programming:
            - subquery/2, subquery/3: Probabilistic queries
            - debugprint/N, error/N: Debugging
            - nocache/2: Cache control
            - scope operations
            - other builtins...''',
                'keywords': ['problog', 'builtin', 'probabilistic', 'subquery', 'evidence', 'cache', 'scope'],
                'summary': 'This section covers ProbLog-specific predicates for handling probabilistic queries, evidence, debugging, and other features unique to probabilistic logic programming.'
            },
            '4.3': {
                'title': 'Available_libraries',
                'description': '''Additional libraries that extend ProbLog functionality:
            - Lists: Standard list operations (member, append, reverse)
            - Aggregate: LDL++ style aggregation (sum, avg, min, max)
            - DB: Database and CSV access
            - Cut: Soft cut for ordered rules
            - Assert: Dynamic fact management
            - Scope: Theory management
            - other libraries...''',
                'keywords': ['library', 'lists', 'aggregate', 'database', 'string', 'cut', 'assert'],
                'summary': 'This section describes various libraries available in ProbLog that provide additional functionality like list operations, aggregation, database access, and more.'
            }
        }

        self.subsection_metadata = {
            '4.1.1': {
                'title': 'Control predicates',
                'description': 'Flow control predicates like conjunction, disjunction, negation, ...',
                'keywords': ['control', 'flow', 'conjunction', 'disjunction', 'fail', 'true', 'false', 'not', 'call', 'once', 'cut', 'conditional statements'],
                'examples': ['P, Q', 'P; Q', 'true/0', 'fail/0', '\\+/1', 'call/1','!/0','P -> Q','P *-> Q']
            },
            '4.1.2': {
                'title': 'Handling Undefined Procedures',
                'description': 'How ProbLog handles undefined predicates',
                'keywords': ['undefined', 'procedure', 'unknown', 'fail'],
                'examples': ['unknown(fail)']
            },
            '4.1.3': {
                'title': 'Message Handling',
                'description': 'Message handling predicates (not supported)',
                'keywords': ['message handling'],
                'examples': []
            },
            '4.1.4': {
                'title': 'Predicates on Terms',
                'description': 'Predicates for term inspection and manipulation',
                'keywords': ['term', 'var', 'atom', 'compound', 'functor', 'arg', 'unification', 'is_list'],
                'examples': ['var/1', 'atom/1', 'compound/1', 'float/1', 'arg/3', 'X = Y', 'X \\= Y','is_list/1','T1 =@= T2']
            },
            '4.1.5': {
                'title': 'Predicates on Atoms',
                'description': 'Atom manipulation predicates (not supported)',
                'keywords': ['atom', 'not supported'],
                'examples': []
            },
            '4.1.6': {
                'title': 'Predicates on Characters',
                'description': 'Character manipulation predicates (not supported)',
                'keywords': ['character', 'not supported'],
                'examples': []
            },
            '4.1.7': {
                'title': 'Comparing Terms',
                'description': 'Term comparison predicates',
                'keywords': ['compare', 'equality', 'sort', 'length', 'ordering'],
                'examples': ['compare/3', 'X == Y', 'X \\== Y', 'X @< Y', 'sort/2', 'length/2']
            },
            '4.1.8': {
                'title': 'Arithmetic',
                'description': 'Arithmetic operations and mathematical functions',
                'keywords': ['arithmetic', 'math', 'calculation', 'plus', 'minus', 'multiply', 'divide', 'exp', 'log', 'sin', 'cos', 'sqrt'],
                'examples': ['X+Y', 'X-Y', 'X*Y', 'X/Y', 'X//Y','exp/1', 'log/1', 'sin/1', 'cos/1', 'X is Y','[X]','random/0']
            },
            '4.1.9': {
                'title': 'Remaining sections',
                'description': 'Other Prolog sections not supported',
                'keywords': ['remaining', 'not supported'],
                'examples': []
            },
            '4.2': {
                'title': 'Problog_specific_builtins',
                'description': 'Special predicates unique to ProbLog for probabilistic logic programming',
                'keywords': ['problog', 'builtin', 'probabilistic', 'subquery', 'evidence', 'cache', 'scope'],
                'examples': ['try_call/N', 'append/3', 'write/N', 'error/N', 'clause/2', 'clause/3']
            },
            '4.3.1': {
                'title': 'Lists',
                'description': 'List manipulation predicates from SWI-Prolog',
                'keywords': ['list', 'member', 'append', 'select', 'reverse', 'sort', 'permutation'],
                'examples': ['member/2', 'append/3', 'select/3', 'reverse/2', 'permutation/2', 'flatten/2']
            },
            '4.3.2': {
                'title': 'Apply',
                'description': 'Higher-order predicates for list processing',
                'keywords': ['apply', 'maplist', 'foldl', 'include', 'exclude', 'partition'],
                'examples': ['maplist/2', 'maplist/3', 'foldl/4', 'include/3', 'exclude/3']
            },
            '4.3.3': {
                'title': 'Cut',
                'description': 'Soft cut implementation for ordered rulesets',
                'keywords': ['cut', 'soft cut', 'ordered', 'rules', 'indexed clauses'],
                'examples': ['cut/2']
            },
            '4.3.4': {
                'title': 'Assert',
                'description': 'Dynamic fact assertion and retraction',
                'keywords': ['assert', 'retract', 'dynamic', 'database', 'assertz', 'retractall'],
                'examples': ['assertz/1', 'retract/1', 'retractall/1']
            },
            '4.3.5': {
                'title': 'Record',
                'description': 'Non-backtrackable storage access',
                'keywords': ['record', 'storage', 'non-backtrackable', 'database'],
                'examples': ['recorda/2', 'recordz/2', 'recorded/2', 'erase/1']
            },
            '4.3.6': {
                'title': 'Aggregate',
                'description': 'LDL++ style aggregation operations',
                'keywords': ['aggregate', 'sum', 'avg', 'min', 'max', 'group by'],
                'examples': ['sum<X>', 'avg<X>', 'min<X>', 'max<X>']
            },
            '4.3.7': {
                'title': 'Collect',
                'description': 'Generalized aggregation with => operator',
                'keywords': ['collect', 'aggregation', 'group by', '=>'],
                'examples': ['(CODE) => GroupBy / AggFunc']
            },
            '4.3.8': {
                'title': 'DB',
                'description': 'SQLite database and CSV file access',
                'keywords': ['database', 'sqlite', 'csv', 'sql', 'data access'],
                'examples': ['sqlite_load/1', 'sqlite_csv/2']
            },
            '4.3.9': {
                'title': 'Scope',
                'description': 'Managing multiple ProbLog theories with scopes',
                'keywords': ['scope', 'theory', 'namespace', 'union', 'conjunction'],
                'examples': ['scope(1):predicate', 'scope1:X; scope2:X']
            },
            '4.3.10': {
                'title': 'String',
                'description': 'String manipulation predicates',
                'keywords': ['string', 'manipulation', 'text processing'],
                'examples': []
            },
            '4.3.11': {
                'title': 'NLP4PLP',
                'description': 'Library for representing and solving probability questions',
                'keywords': ['nlp', 'natural language', 'probability', 'questions'],
                'examples': []
            }
        }
    def split_and_save(self, pdf_path, embedding_function, db_dir):
        loader = PyMuPDFLoader(str(pdf_path))
        page_docs = loader.load()
        
        current_section = None
        current_subsection_key = None
        current_subsection_content = []
        subsection_list = []
        
        for page in page_docs:
            lines = page.page_content.splitlines()
            
            for line in lines[1:-2]:
                # Try to match a section number (4.1, 4.2, 4.3) or a subsection number (4.1.1, 4.1.2, etc.)
                match = re.match(r'^(4\.\d+(?:\.\d+)?)\s', line)
                
                if match:
                    section_key = match.group(1)
                    
                    # Check if it is a main section (4.1, 4.2, 4.3)
                    if section_key in self.section_metadata:
                        # Save the current section content
                        if current_subsection_key and current_subsection_content:
                            doc = Document(
                                page_content="\n".join(current_subsection_content),
                                metadata={
                                    **self.subsection_metadata.get(current_subsection_key, {})
                                }
                            )
                            subsection_list.append(doc)
                        
                        # Save all sub-sections of the current section to the vector database
                        if current_section and subsection_list:
                            print(subsection_list)
                            vector_store = FAISS.from_documents(subsection_list, embedding_function)
                            vector_store.save_local(db_dir, index_name=current_section['title'])
                            subsection_list = []
                        
                        # Start a new chapter
                        current_section = self.section_metadata[section_key]
                        current_subsection_key = None
                        current_subsection_content = []
                    
                    # Check if it is a subsection
                    if section_key in self.subsection_metadata:
                        # Save the current subsection content
                        if current_subsection_key and current_subsection_content:
                            doc = Document(
                                page_content="\n".join(current_subsection_content),
                                metadata={
                                    **self.subsection_metadata.get(current_subsection_key, {})
                                }
                            )
                            subsection_list.append(doc)
                        
                        # Start a new chapter
                        current_subsection_key = section_key
                        current_subsection_content = [line]
                else:
                    # Normal
                    if current_subsection_key:
                        current_subsection_content.append(line)
        
        # handling the last chapter
        if current_subsection_key and current_subsection_content:
            doc = Document(
                page_content="\n".join(current_subsection_content),
                metadata={
                    **self.subsection_metadata.get(current_subsection_key, {})
                }
            )
            subsection_list.append(doc)
        
        # Save the last chapter content
        if current_section and subsection_list:
            print(subsection_list)
            vector_store = FAISS.from_documents(subsection_list, embedding_function)
            vector_store.save_local(db_dir, index_name=current_section['title'])

        print(f"Successfully created 3 vector databases：")
        for section_key, section_info in self.section_metadata.items():
            print(f"  - {section_info['title']}")


    @property
    def vs(self,
        ) -> FAISS:
        """
        Get vector storage object
        returns:
            vector database
        """
        if not self.index_file.exists():
            self.split_and_save(self.problog_official_doc_path, self.embedding_function, self.langda_db_dir)

        return FAISS.load_local(
            self.langda_db_dir, 
            self.embedding_function, 
            index_name=self.index_name,
            allow_dangerous_deserialization=True
        )

    @property
    def get_chapter_metadata(self) -> dict:
        """
        get metadata of a chapter
        """
        for key, value in self.section_metadata.items():
            if value['title'] == self.index_name:
                return value

    def similarity_search(self, query: str, k: int = 5) -> List[Document]:
        """
        Perform similarity search on the local vector library.
        args:
            query: search query text
            k: number of results to return
        returns:
            document list
        """
        return self.vs.similarity_search(query, k=k)


if __name__ == "__main__":
    prolog_builtins = LangdaVectorStore("Prolog_builtins")
    problog_builtins = LangdaVectorStore("Problog_builtins")
    available_libraries = LangdaVectorStore("Available_libraries")
    prolog_builtins.vs
    problog_builtins.vs
    available_libraries.vs

[Document(metadata={'title': 'Control predicates', 'description': 'Flow control predicates like conjunction, disjunction, negation, ...', 'keywords': ['control', 'flow', 'conjunction', 'disjunction', 'fail', 'true', 'false', 'not', 'call', 'once', 'cut', 'conditional statements'], 'examples': ['P, Q', 'P; Q', 'true/0', 'fail/0', '\\+/1', 'call/1', '!/0', 'P -> Q', 'P *-> Q']}, page_content='4.1.1 Control predicates\nSupported:\n• P, Q\n• P; Q\n• true/0\n• fail/0\n• false/0\n• \\+/1\n• not/1\n• call/1\n• call/N (for N up to 9)\n• forall/2\nSpecial:\n• once/1: In ProbLog once/1 is an alias for call/1.\nNot supported:\n• !/0\n• P -> Q\n• P *-> Q\n• repeat\n• incore/1 (use call/1)\n• call_with_args/N (use call/N)\n• if(A,B,C) (use (A,B);(\\+A,C))\n• ignore/1\n• abort/0\n• break/0\n• halt/0\n• halt/1\n• catch/3\n• throw/1\n• garbage_collect/0\n• garbage_collect_atoms/0\n• gc/0\n• nogc/0\n• grow_heap/1\n• grow_stack/1'), Document(metadata={'title': 'Handling Undefined Procedures', 'descripti